In [3]:
from keras.models import Model
from keras.layers import Input, Embedding, merge

import numpy as np
import pickle
import gensim

In [2]:
embedding_matrix = np.load("word2vec_data/embeddings.npy")
word_2_index = pickle.load(open('word2vec_data/word_2_index.pkl', 'rb'))
index_2_word = pickle.load(open('word2vec_data/index_2_word.pkl', 'rb'))
model = gensim.models.Word2Vec.load("word2vec_data/my_word2vec")
wv = model.wv

In [4]:
def cos_similarity(v1, v2):
    numerator = np.dot(v1, v2)
    denominator = np.sqrt(np.sum(np.power(v1, 2))) * np.sqrt(np.sum(np.power(v2, 2)))
    
    return numerator / denominator

In [5]:
def word_similarity(w1, w2):
    idx1 = word_2_index[w1]
    idx2 = word_2_index[w2]

    v1 = embedding_matrix[idx1]
    v2 = embedding_matrix[idx2]
    return cos_similarity(v1, v2)

In [6]:
word_similarity("erkek", "kadın")

0.5309982688681166

In [7]:
# input words - in this case we do sample by sample evaluations of the similarity
valid_word = Input((1,), dtype='int32')
other_word = Input((1,), dtype='int32')

# setup the embedding layer
embeddings = Embedding(input_dim=embedding_matrix.shape[0], output_dim=embedding_matrix.shape[1],
                      weights=[embedding_matrix])

embedded_a = embeddings(valid_word)
embedded_b = embeddings(other_word)
similarity = merge([embedded_a, embedded_b], mode='cos', dot_axes=2)

/home/exastax/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:11: UserWarning: The `merge` function is deprecated and will be removed after 08/2017. Use instead layers from `keras.layers.merge`, e.g. `add`, `concatenate`, etc.
  # This is added back by InteractiveShellApp.init_path()
/home/exastax/anaconda3/lib/python3.6/site-packages/keras/legacy/layers.py:464: UserWarning: The `Merge` layer is deprecated and will be removed after 08/2017. Use instead layers from `keras.layers.merge`, e.g. `add`, `concatenate`, etc.
  name=name)


In [8]:
# create the Keras model
k_model = Model(inputs=[valid_word, other_word], outputs=similarity)

In [9]:
def get_sim(valid_word_idx, vocab_size):
    sim = np.zeros((vocab_size,))
    in_arr1 = np.zeros((1,))
    in_arr2 = np.zeros((1,))
    in_arr1[0,] = valid_word_idx
    for i in range(vocab_size):
        in_arr2[0,] = i
        out = k_model.predict_on_batch([in_arr1, in_arr2])
        sim[i] = out
    return sim

In [10]:
valid_size = 15  # Random set of words to evaluate similarity on.
valid_window = 250  # Only pick dev samples in the head of the distribution.
valid_examples = np.random.choice(valid_window, valid_size, replace=False)

In [11]:
# now run the model and get the closest words to the valid examples
for i in range(valid_size):
    valid_word = index_2_word[valid_examples[i]]
    top_k = 8  # number of nearest neighbors
    sim = get_sim(valid_examples[i], len(wv.vocab))
    nearest = (-sim).argsort()[1:top_k + 1]
    log_str = "Nearest to '%s':" % valid_word
    for k in range(top_k):
        close_word = index_2_word[nearest[k]]
        log_str = '%s %s,' % (log_str, close_word)
    print(log_str)

Nearest to 'fazla': az, fazlası, fazlasını, fazladır, fazlasının, pahalı, ucuza, fazlasına,
Nearest to 'ortaya': karşımıza, açığa, görücüye, sandıktan, çileden, karşıma, ava, karşılarına,
Nearest to 'a': la, ın, ına, tan, takımına, lara, ı, kentine,
Nearest to 'önce': evvel, önceden, öncede, dolmadan, süreyle, önceki, öncesinden, sonraya,
Nearest to 'futbol': basketbol, voleybol, briç, a2, tenis, u15, ragbi, futbolu,
Nearest to '11': 12, 21, 18, 14, 17, 9, 8, 15,
Nearest to 'alan': almıştı, alacak, alıyor, aldı, aldığı, alıyordu, alırken, alarak,
Nearest to 'ayrıca': örneğin, bulunacağı, böylece, kolaylığı, listeleri, zira, değiştirildiği, tavsiyeler,
Nearest to 'sonra': sonrada, sonraki, dolmadan, öncede, önce, itibaren, önceden, sonrasını,
Nearest to 'bunu': bunları, şunu, onu, doğruyu, onları, işimizi, gereğini, zoru,
Nearest to 'e': ten, ine, in, le, teki, lere, kilograma, bine,
Nearest to 'söz': alay, merak, şikayet, eleştiri, tartışma, polemik, sözkonusu, itibar,
Nearest to 'aynı